# Relatório - coletor `savinho-bot`

Alunos:

Sávio Cabral - 20183005980

Lucca Miranda - 20183011736

Para coleta, utilizamos como sementes, URLs de websites de notícias nacionais e internacionais:

a) Principais desafios, decisões e arquitetura utilizada

A arquitetura do coletor consiste em dois principais módulos: a classe `PageFetcher` e a classe `Scheduler`. O Scheduler -ou escalonador- é responsável por coordenar a busca pelas URLs a serem posteriormente coletadas pelo `PageFetcher`. Já o `PageFetcher` recebe a URL do escalonador e é responsável por requisitar o conteúdo da página e processá-lo, enviando novas URLs para o escalonador armazenar e estas serem coletadas posteriormente.


No desenvolvimento do coletor `savinho-bot`, enfrentamos algumas dificuldades na implementação do bot.

In [ ]:
# b) URLs sementes utilizadas

urls = ['https://www.nytimes.com', 'https://www.bbc.com',
        'https://www.g1.globo.com', 'https://www.r7.com/',
        'https://www.em.com.br', 'https://www.estadao.com.br/']

c) Como foi feito, faça referências à classes e métodos do código fonte:
→ Os critérios de exclusão de robôs e quantidade de tempo entre requisições à um mesmo servidor

Inicialmente, a classe `Scheduler` é iniciada com um array de URLs sementes `arr_urls_seeds` e armazena em uma estrutura chave-valor, em que cada chave corresponde a URL raiz de um domínio e o valor uma lista contendo as URLs descobertas durante a coleta.

É importante destacar que o `Scheduler` é o mesmo para diversos `PageFetchers`, isso pois a busca nas páginas pode ser paralelizada para melhorar o desempenho, enquanto que a lista de URLs a serem buscadas deve ser única para todas as `threads`.  Após isso, o `PageFetcher` inicia a busca no método `crawl_new_url`, requisitando uma nova URL do `Scheduler` através do método `get_next_url` e define sua condição de parada o método `has_finished_crawl`, onde é verificado se o limite de páginas já foi alcançado.

Com a URL em mãos, o `PageFetcher` busca por todas as tags `<a>` no HTML da página visando encontrar as URLs, chamando o método `add_new_page` do `Scheduler` para cada página descoberta. A URL será adicionada apenas caso passe pela verificação do método `can_add_page` presente no `Scheduler`, ou seja, a URL só será adicionada caso esta não esteja na lista e a profundidade dela não exceda a especificada previamente.

Outro ponto importante a se destacar é o método `can_fetch_page` do `Scheduler`, que requisita o `/robots` do domínio inicialmente para verificar quais URLs deste domínio podem ser coletadas por robôs ou não, respeitando os critérios definidos por cada domínio. Outra forma de regulamentar o acesso aos servidores é estabelecer um tempo mínimo entre requisições, evitando assim a possível sobrecarga do servidor.

Por fim, ao requisitar a URL e adicionar todas suas URLs filhas no `Scheduler` o processo se repete.

d) O impacto na velocidade de coleta (quantidade de páginas por segundo) ao aumentar o número de threads 1 a 20 de 5 em 5 passos e, logo após, de 30 a 100 com 20 passos. Ao fazer o estudo, colete menos páginas (~100 páginas visitadas pode ser o suficiente).

| Fetchers     | Tempo      |
|----------    |--------    |
| 1            | 0.345      |
| 5            | 1.182      |
| 10           | 23.455     |
| 15           | 23.280     |
| 20           | 22.804     |
| 30           | 23.156     |
| 50           | 26.157     |
| 70           | 28.684     |
| 90           | 28.709     |
| 100          | 28.920     |


Os resultados indicam um aumento no tempo de execução à medida que aumentamos as `threads`, contrariamente do que era esperado. Após uma pesquisa do grupo acreditamos que isso se deve ao fato do `Python Global Interpreter Lock (GIL)` não permitir a execução de `threads` em parelelo para códigos CPU-bound. Todavia, isso explicaria o tempo não diminuir quando o número de `threads` é maior, mas o tempo aumentou de acordo com a tabela acima. O grupo acredita que esse comportamento se deve ao `overhead` da criação e instanciação das classes, que utiliza o processamento da CPU e, por isso, aumenta o tempo de execução. 

https://stackoverflow.com/questions/53721539/multiprocessing-is-slower-than-thread-in-python

https://stackoverflow.com/questions/56372799/python-multiprocessing-code-running-slower-than-single-threaded-one

https://blog.devgenius.io/why-is-multi-threaded-python-so-slow-f032757f72dc

e) Link para a página descrevendo o coletor criado
https://luckened.notion.site/luckened/Especifica-o-coletor-b02b93e51aa840778fd2f225aab62d7b